In [1]:
import os
import torch
import polars as pl
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
from feature.scalers import ranged_scaler
from datetime import datetime, timedelta
#from mpge.rca import mpge_root_cause_diagnosis
warnings.filterwarnings("ignore", category=UserWarning) 

In [2]:
cats_df = pl.read_csv("data/data.csv", separator=",")  
metadata = pl.read_csv('data/metadata.csv',separator=',')
potential_causes = metadata['root_cause'].unique().to_list()

for col in cats_df.columns:
    unique_vals = cats_df[col].n_unique()
    data_type = cats_df[col].dtype
    bad_dtypes = [pl.Date,pl.Datetime,pl.Utf8]
    if ((unique_vals >= 50) & (data_type not in bad_dtypes) ):
        cats_df = cats_df.with_columns(ranged_scaler(cats_df[col]))
    else:
        continue

In [3]:
cats_df = cats_df.with_columns(
    pl.col('timestamp').str.to_datetime("%Y-%m-%d %H:%M:%S"),
    pl.Series("entity_id",range(len(cats_df)))
)
cats_rows_list = metadata.rows(named=True)

In [4]:
cats_rows_list = metadata.rows(named=True)
cats_df.head()

timestamp,aimp,amud,arnd,asin1,asin2,adbr,adfl,bed1,bed2,bfo1,bfo2,bso1,bso2,bso3,ced1,cfo1,cso1,y,category,entity_id
datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
2023-01-01 00:00:00,0.0,0.142857,-0.5,-4.1078e-14,2.0428e-14,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180547,-0.507953,-0.716059,-0.774361,0.100389,-0.186623,0.0,0.0,0
2023-01-01 00:00:01,0.0,0.142857,-0.495998,0.00002,0.0002,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.18054,-0.507953,-0.716059,-0.774361,0.100389,-0.186618,0.0,0.0,1
2023-01-01 00:00:02,0.0,0.142857,-0.486172,0.00004,0.0004,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180519,-0.507953,-0.716059,-0.774361,0.10039,-0.186604,0.0,0.0,2
2023-01-01 00:00:03,0.0,0.142857,-0.463453,0.00006,0.0006,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180484,-0.507953,-0.716059,-0.774361,0.100391,-0.18658,0.0,0.0,3
2023-01-01 00:00:04,0.0,0.142857,-0.444095,0.00008,0.0008,0.0,0.0,-0.32802,-0.369237,-0.738163,-0.767181,-0.180437,-0.507953,-0.716059,-0.774361,0.100393,-0.186548,0.0,0.0,4


In [5]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from cr_vae.cgru_error import CRVAE, VRAE4E, train_phase1, train_phase2
import scipy.io
torch.set_num_threads(10)  # Limit to 4 threads
torch.set_num_interop_threads(5)

In [ ]:
new_metadata = []
device = torch.device('cpu')
correct = 0
for i, row in enumerate(cats_rows_list):
    num_iter = i+1
    potential_causes = metadata['root_cause'].unique().to_list()

    start_time = datetime.strptime(row['start_time'],"%Y-%m-%d %H:%M:%S")
    end_time = datetime.strptime(row['end_time'],"%Y-%m-%d %H:%M:%S")
    anomaly = eval(row['affected'])[0]
    root_cause = row['root_cause']
    potential_causes.append(anomaly)
    mod_df = cats_df.filter( (pl.col('timestamp')>= start_time) & (pl.col('timestamp') <= end_time))
    mod_df = mod_df.drop(['timestamp',"y",'category','entity_id'])
    mod_data = mod_df.to_numpy()
    dim = mod_data.shape[-1]
    GC = np.zeros([dim,dim])
    for j in range(dim):
        GC[j,j] = 1
        if j!=0:
            GC[j,j-1] = 1
    X = torch.tensor(mod_data[np.newaxis], dtype=torch.float32, device=device)
    full_connect = np.ones(GC.shape)
    cgru = CRVAE(X.shape[-1], full_connect, hidden=64)#.cuda(device=device)
    vrae = VRAE4E(X.shape[-1], hidden=64)#.cuda(device=device)
    train_loss_list = train_phase1(
    cgru, X, context=20, lam=0.1, lam_ridge=0, lr=5e-2, max_iter=250,
    check_every=1000,verbose=0)#0.1
    GC_est = cgru.GC().float().cpu().data.numpy()
    adj_df = pd.DataFrame(GC_est,index=mod_df.columns,columns=mod_df.columns)
    causes = adj_df.loc[potential_causes][anomaly].sort_values(ascending=False)
    potential_cause1 = causes.index[0]
    potential_cause2 = causes.index[1]
    potential_cause3 = causes.index[2]
    if causes[root_cause] == 1:
        row['cause_1'] = 1
        print('*********Correct*********')
        correct +=1
    else:
        row['cause_1']=0
        print('*********Incorrect*********')
    new_metadata.append(row)
    print('Model Accuracy: '+str(correct/num_iter))

----------Iter = 0----------
Loss = 0.140754
KL = 0.786345
Variable usage = 100.00%
*********Correct*********
Model Accuracy: 1.0
----------Iter = 0----------
Loss = 0.195013
KL = 0.874375
Variable usage = 100.00%
